# How to save a LightCurve in FITS format?

Once you have detrended or altered a lightcurve in some way, you may want to save it as a FITS file. This allows you to easily share the file with your collaborators or submit your lightcurves as a [MAST High Level Science Product](https://archive.stsci.edu/hlsp/hlsp_guidelines.html) (HLSP). Lightkurve provides a [to_fits()](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.to_fits.html?highlight=to_fits#lightkurve.LightCurve.to_fits) method which will easily convert your [LightCurve](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.html?highlight=lightcurve#lightkurve.LightCurve) object into a fits file.

Below is a brief demostration showing how `to_fits()` works.

Note: if you are considering contributing a HLSP you may want to read the [guidelines](https://archive.stsci.edu/hlsp/hlsp_guidelines_timeseries.html) for contributing fits files. These include which fits headers are required/suggested for your HLSP to be accepted.

## Example: editing and writing a lightcurve

First we'll obtain a random Kepler lightcurve from MAST.

In [ ]:
from lightkurve import search_lightcurve
lc = search_lightcurve('KIC 757076', author="Kepler", quarter=3).download()

Now we'll make some edits to the lightcurve. Below we use the PDCSAP flux from MAST, remove NaN values using the [.remove_nans()](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.remove_nans.html?highlight=remove_nans) function, and clip out any outliers using the [.remove_outliers()](https://docs.lightkurve.org/reference/api/lightkurve.LightCurve.remove_outliers.html?highlight=remove_outliers).

In [ ]:
lc = lc.remove_nans().remove_outliers()
lc.scatter();

Now we can use the `to_fits` method to save the lightcurve to a file called *output.fits*.

In [ ]:
lc.to_fits(path='demo-lightcurve.fits', overwrite=True)

Let's take a look at the file and check that it behaved as we expect

In [ ]:
from astropy.io import fits
hdu = fits.open('demo-lightcurve.fits')
type(hdu)

In [ ]:
hdu.info()

`hdu` is a set of astropy.io.fits objects, which is what we would expect. Lets take a look at the header of the first extension.

In [ ]:
hdu[0].header

Looks like it has all the correct information about the target. What about the second extension?

In [ ]:
hdu[1].header

This extension has 7 columns, `TIME`, `FLUX`, `FLUX_ERR`, `SAP_QUALITY`, `CADENCENO`, `MOM_CENTR1`, and `MOM_CENTR2`.  What if we wanted to add new keywords to our fits file?  HLSP products require some extra keywords. Let's add some keywords to explain who made the data, and what our HLSP is. 

In [ ]:
lc.to_fits(path='demo-lightcurve.fits',
           overwrite=True,
           HLSPLEAD='Kepler/K2 GO office',
           HLSPNAME='TUTORIAL',
           CITATION='HEDGES2018')

In [ ]:
hdu = fits.open('demo-lightcurve.fits')

In [ ]:
hdu[0].header

Now our new keywords are included in the primary header! What about if we want to add more **data columns** to our fits file? We can simply add data columns in the same way. Let's add the data quality to our fits file.

In [ ]:
demo_vector = lc.fold(period=1.23456789).phase
demo_vector

In [ ]:
lc.to_fits(path='demo-lightcurve.fits',
           overwrite=True,
           HLSPLEAD='Kepler/K2 GO office',
           HLSPNAME='TUTORIAL',
           CITATION='HEDGES2018',
           DEMO_COLUMN=demo_vector)

In [ ]:
hdu = fits.open('demo-lightcurve.fits')

In [ ]:
hdu[1].data.columns

The vector is populated as a data column in the HDU extension.  Once all your lightcurves are saved as fits files and you have a README file, you can consider submitting your data products to MAST.